# Apply VADER Model

The purpose of this notebook is to apply the VADER model to the cleaned AirBNB and tripadvisor data, and combine all of the data into one master file.

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#### Change Setting Variables

Change the variable below in order change how the notebook functions.

In [2]:
zipcode_column_name = "zip_codes" #The name of the zipcode column within the TripAdvisor file
COVID_column_name = "COVID" #The name of the COVID column within the TripAdvisor file

testing = False #Set to True if testing the script. Set to False if running the script fully.
airbnb_directory = "data/AirBNB" #The directory where all of the cleaned AirBNB files are
tripadvisor_file = "data/trip_advisor_revised.csv.gz" #The path to the cleaned TripAdvisor data

#### Changing the TripAdvisor Layout and Running Applying VADER Model

In [3]:
#Reading in data
if testing == True:
    tripadvisor_df = pd.read_csv(tripadvisor_file, compression = "gzip", nrows = 100000)
    tripadvisor_df[zipcode_column_name] = np.nan
    tripadvisor_df[COVID_column_name] = np.nan
else:
    tripadvisor_df = pd.read_csv(tripadvisor_file, compression = "gzip")

In [4]:
tripadvisor_df.shape

(1824732, 6)

In [5]:
tripadvisor_df.head()

,City,zip_codes,Date,Price,Review,COVID
0,asheville,28803,2021-02-28,$$$,Clean with Spectacular Service This hotel was ...,Post-
1,asheville,28803,2021-01-31,$$$,Awesome Hotel! My boyfriend and I had a weeken...,Post-
2,asheville,28803,2021-02-28,$$$,"Great hotel Decided on a trip to Asheville, it...",Post-
3,asheville,28803,2021-02-28,$$$,Great hotel We loved this hotel! Beautifully d...,Post-
4,asheville,28803,2021-02-28,$$$,"Essence of ""Southern Hospitality""! This is a w...",Post-


In [6]:
#Renaming columns to match AirBNB column names
tripadvisor_df = tripadvisor_df.rename(columns = {"City":"city", "Date":"date", "Price":"price", "Review":"comments",zipcode_column_name:"zipcode", COVID_column_name:"COVID"})

In [7]:
#Creating columns that exist in AirBNB but do not exist in TripAdvisor
new_columns = ["listing_id", "neighborhood", "property", "room", "rating"]
for column in new_columns:
    tripadvisor_df[column] = np.nan

In [8]:
#Reordering columns to match AirBNB column order
column_names = ["listing_id", "neighborhood", "price", "property", "room", "rating", "zipcode", "date", "comments", "COVID", "city"]
tripadvisor_df = tripadvisor_df[column_names]

In [9]:
#Adding a source column so we can easily separate out TripAdvisor data
tripadvisor_df["source"] = "TripAdvisor"
tripadvisor_df.head()

,listing_id,neighborhood,price,property,room,rating,zipcode,date,comments,COVID,city,source
0,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,Clean with Spectacular Service This hotel was ...,Post-,asheville,TripAdvisor
1,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-01-31,Awesome Hotel! My boyfriend and I had a weeken...,Post-,asheville,TripAdvisor
2,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,"Great hotel Decided on a trip to Asheville, it...",Post-,asheville,TripAdvisor
3,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,Great hotel We loved this hotel! Beautifully d...,Post-,asheville,TripAdvisor
4,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,"Essence of ""Southern Hospitality""! This is a w...",Post-,asheville,TripAdvisor


In [10]:
#Setting up VADER
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tom/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
#Setting up function to apply VADER model
def create_polarity_scores(df):
    neg = []
    neu = []
    pos = []
    compound = []
    error = []
    for index, row in df.iterrows():
        try:
            pol_dict = sia.polarity_scores(row["comments"])
            neg.append(pol_dict["neg"])
            neu.append(pol_dict["neu"])
            pos.append(pol_dict["pos"])
            compound.append(pol_dict["compound"])
            error.append(0)
        except AttributeError:
            neg.append(np.nan)
            neu.append(np.nan)
            pos.append(np.nan)
            compound.append(np.nan)
            error.append(1)
    return neg, neu, pos, compound, error

In [12]:
#Applying VADER model
%time neg, neu, pos, compound, error = create_polarity_scores(tripadvisor_df)

CPU times: user 33min 50s, sys: 23.5 s, total: 34min 13s
Wall time: 34min 13s


In [13]:
tripadvisor_df["neg"] = neg
tripadvisor_df["neu"] = neu
tripadvisor_df["pos"] = pos
tripadvisor_df["compound"] = compound
tripadvisor_df["vader_error"] = error

In [14]:
#Setting up dictionary to report out VADER errors
vader_errors_dict = {"tripadvisor":tripadvisor_df["vader_error"].sum()}

In [15]:
tripadvisor_df.head()

,listing_id,neighborhood,price,property,room,rating,zipcode,date,comments,COVID,city,source,neg,neu,pos,compound,vader_error
0,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,Clean with Spectacular Service This hotel was ...,Post-,asheville,TripAdvisor,0.009,0.732,0.259,0.9964,0
1,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-01-31,Awesome Hotel! My boyfriend and I had a weeken...,Post-,asheville,TripAdvisor,0.000,0.559,0.441,0.9782,0
2,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,"Great hotel Decided on a trip to Asheville, it...",Post-,asheville,TripAdvisor,0.000,0.606,0.394,0.9843,0
3,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,Great hotel We loved this hotel! Beautifully d...,Post-,asheville,TripAdvisor,0.034,0.528,0.438,0.9769,0
4,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,"Essence of ""Southern Hospitality""! This is a w...",Post-,asheville,TripAdvisor,0.000,0.728,0.272,0.9848,0


In [16]:
#Saving the initial csv
tripadvisor_df.to_csv("data/vaderoutput.csv.gz", compression = 'gzip', index = False)

In [17]:
#Releasing the tripadvisor df from memory
del tripadvisor_df

#### Applying the VADER model to the AirBNB files

In [18]:
#Setting up column name list to test that names are consistent
column_names = ["listing_id", "neighborhood", "price", "property", "room", "rating", "zipcode", "date", "comments", "COVID"]
error_dict = {}

In [19]:
for f in os.listdir(airbnb_directory):
    if f != ".DS_Store" and "gz" in f:
        print(f)
        #Reading in file
        if testing == True:
            df = pd.read_csv(os.path.join(airbnb_directory, f), compression = "gzip", nrows = 100000)
        else:
            df = pd.read_csv(os.path.join(airbnb_directory, f), compression = "gzip")
        
        try:
            #Correctly arranging column names if necessary
            df = df[column_names]
            #Processing files
            print("Number of Rows: %s" % len(df))
            city = f.split(".")[0]
            df["city"] = city
            df["source"] = "AirBNB"
            neg, neu, pos, compound, error = create_polarity_scores(df)
            df["neg"] = neg
            df["neu"] = neu
            df["pos"] = pos
            df["compound"] = compound
            df["vader_error"] = error
            vader_errors = df["vader_error"].sum()
            vader_errors_dict[f] = vader_errors
            print(df.head())
            print("\n")
            df.to_csv("data/vaderoutput.csv.gz", compression = 'gzip', index = False, mode = "a", header = False)
        except Exception as e:
            #If there is an error, adding it to the error dict for later processing
            error_dict[f] = e

Seattle.csv.gz


/home/tom/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Number of Rows: 349087
   listing_id neighborhood    price      property             room  rating  \
0        2318      Madrona  $296.00  Entire house  Entire home/apt   100.0   
1        2318      Madrona  $296.00  Entire house  Entire home/apt   100.0   
2        2318      Madrona  $296.00  Entire house  Entire home/apt   100.0   
3        2318      Madrona  $296.00  Entire house  Entire home/apt   100.0   
4        2318      Madrona  $296.00  Entire house  Entire home/apt   100.0   

  zipcode        date                                           comments  \
0     NaN  2018-02-20  Lovely, sunny house.  Fully equipped kitchen. ...   
1     NaN  2018-02-25  This house made me want to move to Madrona :)....   
2     NaN  2018-03-28  Casa Madrona was perfect for our family of 5! ...   
3     NaN  2018-04-04  Megan's house is spacious, attractive, ultra-c...   
4     NaN  2018-04-22  While in Seattle for a couple weeks, I rented ...   

  COVID     city  source  neg    neu    pos  compou

/home/tom/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Number of Rows: 90933
   listing_id          neighborhood   price               property  \
0       11879  Unincorporated Areas  $75.00  Private room in house   
1       11879  Unincorporated Areas  $75.00  Private room in house   
2       11879  Unincorporated Areas  $75.00  Private room in house   
3       11879  Unincorporated Areas  $75.00  Private room in house   
4       11879  Unincorporated Areas  $75.00  Private room in house   

           room  rating zipcode        date  \
0  Private room    97.0     NaN  2018-06-01   
1  Private room    97.0     NaN  2018-06-12   
2  Private room    97.0     NaN  2018-06-20   
3  Private room    97.0     NaN  2018-07-23   
4  Private room    97.0     NaN  2018-08-13   

                                            comments COVID       city  source  \
0  If you're staying in Aptos, Steve's house is i...  Pre-  SantaCruz  AirBNB   
1  Sunny room was perfect for my needs. Relaxed a...  Pre-  SantaCruz  AirBNB   
2  Steven's home is comfortable

...

Number of Rows: 1223810
   listing_id neighborhood    price   property             room  rating  \
0        2595      Midtown  $225.00  Apartment  Entire home/apt    94.0   
1        2595      Midtown  $225.00  Apartment  Entire home/apt    94.0   
2        2595      Midtown  $225.00  Apartment  Entire home/apt    94.0   
3        2595      Midtown  $225.00  Apartment  Entire home/apt    94.0   
4        2595      Midtown  $225.00  Apartment  Entire home/apt    94.0   

  zipcode        date                                           comments  \
0   10018  2018-04-29  We really enjoyed our stay at Jennifer’s apart...   
1   10018  2018-05-18  This apartment is convenient and functional.  ...   
2   10018  2018-05-27  The Midtown Castle is a beautiful place to sta...   
3   10018  2018-06-06  Jennifer is very nice and responsive. The loca...   
4   10018  2018-09-27  Awesome location, spotless, wonderfully accomm...   

  COVID     city  source    neg    neu    pos  compound  vader_error

/home/tom/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Number of Rows: 164251
   listing_id neighborhood    price   property             room  rating  \
0        3781  East Boston  $125.00  Apartment  Entire home/apt    99.0   
1        3781  East Boston  $125.00  Apartment  Entire home/apt    99.0   
2        3781  East Boston  $125.00  Apartment  Entire home/apt    99.0   
3        3781  East Boston  $125.00  Apartment  Entire home/apt    99.0   
4        3781  East Boston  $125.00  Apartment  Entire home/apt    99.0   

  zipcode        date                                           comments  \
0   02128  2018-07-02       Great place, amazing city, fantastic host!!!   
1   02128  2019-02-22  My partner and I stayed at Frank's apartment f...   
2   02128  2019-12-21  I liked very much this apartment rented by Fra...   
3   02128  2020-09-03  I stayed at Frank's apartment for three months...   
4   02128  2020-12-20  Wonderful 1 bedroom with a great location! The...   

   COVID    city  source  neg    neu    pos  compound  vader_error  


/home/tom/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Number of Rows: 116652
   listing_id  neighborhood    price property             room  rating  \
0       15373  Narragansett  $415.00    House  Entire home/apt     NaN   
1       15373  Narragansett  $415.00    House  Entire home/apt     NaN   
2       15373  Narragansett  $415.00    House  Entire home/apt     NaN   
3       21714     Pawtucket  $168.00     Loft  Entire home/apt    87.0   
4       21714     Pawtucket  $168.00     Loft  Entire home/apt    87.0   

  zipcode        date                                           comments  \
0    2882  2020-07-21         The place clean and the location is great.   
1    2882  2020-08-14  We had a great time at this cottage. It’s larg...   
2    2882  2020-08-28  Amazing spot. Quick walk to the beach. Lots of...   
3    2863  2018-11-23                 A great host and a beautiful place   
4    2863  2018-11-25  I usually don’t write reviews for anything, bu...   

   COVID         city  source    neg    neu    pos  compound  vader_error  

#### Error Checking

Below are the list of AirBNB files that had errors raised. These files will have to be processed independently, and appended to the output at a later time.

In [20]:
print("Errors in %s files" % len(error_dict))
print(error_dict)

Errors in 1 files
{'Dallas.csv.gz': KeyError("['zipcode'] not in index")}


Below are the list of files that had VADER errors greater than 0. If the number of VADER errors for a given file is large we may have to find the root cause.

In [21]:
file_count = 0
for f in sorted(vader_errors_dict.keys()):
    error_count = vader_errors_dict[f]
    if error_count > 0:
        print("%s - %s" % (f, error_count))
        file_count+= 1

if file_count == 0:
    print("No VADER errors")

ClarkCounty.csv.gz - 1
Oakland.csv.gz - 9


Showing output file format at top of file and near end of file

In [22]:
output_file = pd.read_csv("data/vaderoutput.csv.gz", compression = 'gzip', nrows = 5)

In [23]:
output_file.head()

,listing_id,neighborhood,price,property,room,rating,zipcode,date,comments,COVID,city,source,neg,neu,pos,compound,vader_error
0,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,Clean with Spectacular Service This hotel was ...,Post-,asheville,TripAdvisor,0.009,0.732,0.259,0.9964,0
1,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-01-31,Awesome Hotel! My boyfriend and I had a weeken...,Post-,asheville,TripAdvisor,0.000,0.559,0.441,0.9782,0
2,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,"Great hotel Decided on a trip to Asheville, it...",Post-,asheville,TripAdvisor,0.000,0.606,0.394,0.9843,0
3,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,Great hotel We loved this hotel! Beautifully d...,Post-,asheville,TripAdvisor,0.034,0.528,0.438,0.9769,0
4,NaN,NaN,$$$,NaN,NaN,NaN,28803,2021-02-28,"Essence of ""Southern Hospitality""! This is a w...",Post-,asheville,TripAdvisor,0.000,0.728,0.272,0.9848,0


In [24]:
column_names = output_file.columns

In [25]:
if testing == True:
    skip = 200000
else:
    skip = 8460000

In [26]:
output_file = pd.read_csv("data/vaderoutput.csv.gz", compression = 'gzip', nrows = 5, skiprows = skip)

In [27]:
output_file.columns = column_names

In [28]:
output_file.head()

,listing_id,neighborhood,price,property,room,rating,zipcode,date,comments,COVID,city,source,neg,neu,pos,compound,vader_error
0,27923937,"Dupont Circle, Connecticut Avenue/K Street",$110.00,Apartment,Entire home/apt,95.0,20009,2019-08-25,This is a great location in DC. Dupont Circle ...,Pre-,WashingtonDC,AirBNB,0.0,0.732,0.268,0.9313,0
1,27923937,"Dupont Circle, Connecticut Avenue/K Street",$110.00,Apartment,Entire home/apt,95.0,20009,2019-08-26,The space is adorable and the location is amaz...,Pre-,WashingtonDC,AirBNB,0.0,0.490,0.510,0.8070,0
2,27923937,"Dupont Circle, Connecticut Avenue/K Street",$110.00,Apartment,Entire home/apt,95.0,20009,2019-09-02,This unit is in a GREAT location. Can’t get an...,Pre-,WashingtonDC,AirBNB,0.0,0.869,0.131,0.7034,0
3,27923937,"Dupont Circle, Connecticut Avenue/K Street",$110.00,Apartment,Entire home/apt,95.0,20009,2019-09-05,"Stylish, comfortable, centrally located - grea...",Pre-,WashingtonDC,AirBNB,0.0,0.510,0.490,0.8268,0
4,27923937,"Dupont Circle, Connecticut Avenue/K Street",$110.00,Apartment,Entire home/apt,95.0,20009,2019-09-06,"Great location, very clean. \nFlexible check-i...",Pre-,WashingtonDC,AirBNB,0.0,0.193,0.807,0.9113,0
